In [1]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical
from keras.preprocessing.image import load_img, img_to_array
from keras import optimizers
from keras.models import Model
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras.models import model_from_json
from keras.callbacks import ModelCheckpoint
from keras.preprocessing import image

import os
import numpy as np
import pandas as pd
import random
import glob
import math

from PIL import ImageFile
from matplotlib.pyplot import imshow

Using TensorFlow backend.


In [2]:
%matplotlib inline

# Loading the data

In [5]:
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [6]:
def get_im(path):
    # Load as grayscale
    img = image.load_img(path, target_size=(150, 150))
    # Reduce size
    resized = image.img_to_array(img)
    return resized

In [7]:
# Get Data
def load_train():
    X_train = []
    y_train = []
    print('Read train images')
    thr = math.floor(1485/10)
    for j in range(3):
        print('Load folder Type_{}'.format(j))
        path = os.path.join('data', 'train', 'Type_' + str(j), '*.jpg')
        files = glob.glob(path)
        for fl in files:
            img = get_im(fl)
            X_train.append(img)
            y_train.append(j)
            total += 1
            if total%thr == 0:
            print('Read {} images from 1485'.format(total))

    return X_train, y_train

In [8]:
def load_test():
    print('Read test images')
    path = 'data/test/unknown/*.jpg'
    files = glob.glob(path)
    X_test = []
    X_test_id = []
    total = 0
    thr = math.floor(len(files)/10)
    for fl in files:
        flbase = os.path.basename(fl)
        img = get_im(fl)
        X_test.append(img)
        X_test_id.append(flbase)
        total += 1
        if total%thr == 0:
            print('Read {} images from {}'.format(total, len(files)))

    return X_test.shape

In [66]:
load_test()
#get_im('data/test/unknown/2.jpg')

Read test images
Read 51 images from 512
Read 102 images from 512
Read 153 images from 512
Read 204 images from 512
Read 255 images from 512
Read 306 images from 512
Read 357 images from 512
Read 408 images from 512
Read 459 images from 512
Read 510 images from 512


([array([[[  43.,   46.,   39.],
          [  45.,   46.,   40.],
          [  41.,   44.,   37.],
          ..., 
          [  33.,   23.,   22.],
          [  32.,   22.,   21.],
          [  35.,   24.,   22.]],
  
         [[  44.,   45.,   37.],
          [  44.,   45.,   39.],
          [  40.,   43.,   36.],
          ..., 
          [  32.,   22.,   20.],
          [  37.,   27.,   25.],
          [  34.,   23.,   21.]],
  
         [[  47.,   48.,   40.],
          [  44.,   46.,   33.],
          [  45.,   49.,   35.],
          ..., 
          [  31.,   21.,   19.],
          [  39.,   28.,   26.],
          [  42.,   32.,   30.]],
  
         ..., 
         [[ 138.,  101.,  119.],
          [ 143.,  105.,  126.],
          [ 144.,  110.,  127.],
          ..., 
          [ 155.,  152.,  169.],
          [ 151.,  150.,  166.],
          [ 150.,  149.,  165.]],
  
         [[ 138.,  101.,  118.],
          [ 140.,  103.,  121.],
          [ 144.,  107.,  124.],
          ...,

In [ ]:
load_train()

Read train images
Load folder Type_0
Load folder Type_1


In [3]:
train_data_dir = 'data/train'
validation_data_dir = 'data/valid'
test_data_dir = 'data/test'
results_path = 'results/predictions'
results_name = 'predictions.csv'

In [4]:
# dimensions of our images.
img_width, img_height = 150, 150

# Found 24198 images belonging to 2 classes.
# Found 802 images belonging to 2 classes.
nb_train_samples = 1037
nb_validation_samples = 444

epochs = 20
batch_size = 16

In [5]:
base_model = applications.VGG16(include_top=False, weights='imagenet', input_tensor=None, input_shape=(img_width, img_height, 3))
print('Model Loaded')

Model Loaded


In [6]:
top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(3, activation='sigmoid'))

In [7]:
# add the model on top of the convolutional base
model = Model(input=base_model.input, output=top_model(base_model.output))

C:\Users\Flo\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("se...)`
  


In [8]:
# set the first 25 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
for layer in model.layers[:25]:
    layer.trainable = False

In [9]:
# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

In [20]:

print(X_train.shape, Y_train.shape)

(2346, 150, 150, 3) (2346, 3)


In [21]:
# checkpoint
filepath="vgg-weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [22]:
# See: https://github.com/fchollet/keras/issues/5475
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [24]:
model.fit(X_train, Y_train, batch_size=16, epochs=2, callbacks=callbacks_list, validation_split=0.33, shuffle=True, initial_epoch=0)

Train on 1571 samples, validate on 775 samples
Epoch 1/2


KeyError: '[1079  805  831  303 1550   45  887 1303 1138  709 1115  470 1416  355  868\n 1486  696 1380 1070   66 1442  257 1291 1190 1170 1290  522 1379 1007 1436\n  673 1350] not in index'

In [13]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
validation_steps=nb_validation_samples // batch_size,
callbacks=callbacks_list)
print('Finished training!')

Epoch 1/50


Exception in thread Thread-6:
Traceback (most recent call last):
  File "C:\Users\Flo\Anaconda3\envs\tensorflow\lib\threading.py", line 914, in _bootstrap_inner
    self.run()
  File "C:\Users\Flo\Anaconda3\envs\tensorflow\lib\threading.py", line 862, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Flo\Anaconda3\envs\tensorflow\lib\site-packages\keras\engine\training.py", line 612, in data_generator_task
    generator_output = next(self._generator)
  File "C:\Users\Flo\Anaconda3\envs\tensorflow\lib\site-packages\keras\preprocessing\image.py", line 727, in __next__
    return self.next(*args, **kwargs)
  File "C:\Users\Flo\Anaconda3\envs\tensorflow\lib\site-packages\keras\preprocessing\image.py", line 963, in next
    x = self.image_data_generator.standardize(x)
  File "C:\Users\Flo\Anaconda3\envs\tensorflow\lib\site-packages\keras\preprocessing\image.py", line 486, in standardize
    x = self.preprocessing_function(x)
  File "C:\Users\Flo\Anaconda3\envs\tensorflow

ValueError: output of generator should be a tuple `(x, y, sample_weight)` or `(x, y)`. Found: None

In [ ]:
# serialize model to JSON
model_json = model.to_json()
with open("model_xception.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model_xception.h5")
print("Saved model to disk")

In [ ]:
# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()